# Hierarchy Json Pattern for Config Management

[![](https://img.shields.io/badge/STAR_Me_on_GitHub!--None.svg?style=social)](https://github.com/MacHu-GWU/config_patterns-project)
[![](https://img.shields.io/pypi/v/config_patterns.svg)](https://pypi.python.org/pypi/config_patterns)
[![](https://img.shields.io/badge/Link-Submit_Issue-blue.svg)](https://github.com/MacHu-GWU/config_patterns-project/issues)

## What is Hierarchy JSON

In object-oriented programming, the inheritance hierarchy is a pattern where child objects inherit attributes and methods from parent objects. Similarly, in configuration management, the global configuration often acts as the default value, allowing for the possibility of overriding specific values in environment-specific configurations. For example,if you want to manage the configuration of two servers, a development server and a production server, you may wish to set a global default memory allocation of 2GB. If the memory allocation is not specified on the development server, it will use the default value of 2GB; otherwise, it will use the defined value."

``config_pattern`` provides a feature to declare your hierarchy JSON data for configuration efficiently and derive the final representation of the data.

In [1]:
import json
from rich import print as rprint

def jprint(data: dict):
    rprint(json.dumps(data, indent=4))

## Basic Example

In the following example, there’s a meta field ``_shared`` in the root level of the config file. It is a powerful inheritance hierarchy mechanism to specify config values. The ``_shared`` field is a key value pairs of JSON path notation and it’s value. It set a global default memory allocation of 2GB.

You can also use ``config_pattern.patterns.hierarchy.SHARED`` constant variable to replace ``"_shared"``.

```python
from config_patterns.patterns.hierarchy import SHARED

config_raw = {
    SHARED: {
        "*.memory": 2
    },
    "dev": {
    },
    "prod": {
        "memory": 8
    }
}
```

In [2]:
from config_patterns.patterns.hierarchy.api import SHARED, apply_shared_value

config_data = {
    "_shared": {
        "*.memory": 2
    },
    "dev": {
    },
    "prod": {
        "memory": 8
    }
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "memory": 2
    },
    "prod": {
        "memory": 8
    }
}

## JSON Path Notation Example

``config_patterns`` library use a simplified version of [JSON Path notation](https://docs.oracle.com/cd/E60058_01/PDF/8.0.8.x/8.0.8.0.0/PMF_HTML/JsonPath_Expressions.htm). The JSON path travels from the same level of where the ``SHARED`` meta field is defined. The ``*`` notation matches all fields at the same level (Except the ``SHARED`` field). A ``.`` dot notation visits the child field of the current node.

In [3]:
config_data = {
    SHARED: {
        "*.name": "alice",
        "*.contact.email": "alice@email.com",
    },
    "dev": {
        "contact": {},
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        },
    },
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "contact": {
            "email": "alice@email.com"
        },
        "name": "alice"
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        }
    }
}

## Match a Field in A List of Objects

Dot notation can also access the field in a list of object. In the following example, the ``.port`` in the ``*.databases.port`` JSON path represents the ``port`` field in the database object in ``*.databases`` field.

In [4]:
config_data = {
    SHARED: {
        "*.databases.port": 5432
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com"},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com"},
        ],
    },
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 5432
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 5432
            },
            {
                "host": "db4.com",
                "port": 5432
            }
        ]
    }
}

## Enumerate The Fields You want to Match

Sometimes you don't want to match all fields using ``*`` notation. You could enumerate the fields you want to access.

I knew that the best way to implement this is using "Filter expression". However, this feature is not implemented in ``1.X.Y``.

In [5]:
config_data = {
    SHARED: {
        "dev.databases.port": 5432,
        "prod.databases.port": 3306,
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com", "port": 0},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com", "port": 1},
        ],
    },
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 0
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 3306
            },
            {
                "host": "db4.com",
                "port": 1
            }
        ]
    }
}

## Evaluation Order within ``SHARED`` at the Same Level

You could have multiple JSON Path notations that match the same field. The value defined by the first JSON Path will overwrite others. Since [The insertion-order preservation nature of dict objects has been declared to be an official part of the Python language spec from Python3.7](https://docs.python.org/3/whatsnew/3.7.html), The declaration order in JSON or Python Dict will be the same as the evaluation order.

In the following example, we set a global default 4GB memory for ``dev.*.memory``, and a global default memory 2GB for any server.

In [6]:
config_data = {
    SHARED: {
        "dev.*.memory": 4,
        "*.*.memory": 2,
    },
    "dev": {
        "app1": {},
        "app2": {"memory": 8},
    },
    "prod": {
        "app1": {},
        "app2": {"memory": 16},
    },
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "app1": {
            "memory": 4
        },
        "app2": {
            "memory": 8
        }
    },
    "prod": {
        "app1": {
            "memory": 2
        },
        "app2": {
            "memory": 16
        }
    }
}

## Evaluation Order in Different Level

"You can have the ``SHARED`` meta field at any level of your hierarchy. The closer it is to the root, the more it is considered "global". The deeper it is, the more it is considered "specific". In other word, the ``SHARED`` meta field at deeper level will overwrite others.

This pattern can also be implemented by declaring them in a specific order within the same ``SHARED`` meta field. However, the order of definition is crucial (see the previous section). Personally, I recommend creating a separate ``SHARED`` meta field at a deeper level due to the principle of 'Zen of Python - Explicit is Better than Implicit'.

In [7]:
config_data = {
    SHARED: {
        "*.*.memory": 2,
    },
    "dev": {
        SHARED: {
            "*.memory": 4
        },
        "app1": {},
        "app2": {"memory": 8},
    },
    "prod": {
        "app1": {},
        "app2": {"memory": 16},
    },
}
apply_shared_value(config_data)
print("final config data:")
jprint(config_data)

final config data:


{
    "dev": {
        "app1": {
            "memory": 4
        },
        "app2": {
            "memory": 8
        }
    },
    "prod": {
        "app1": {
            "memory": 2
        },
        "app2": {
            "memory": 16
        }
    }
}

## Summary

Hierarchy Json Pattern is very powerful for config management. To see an example using this pattern in real production-ready projects, please refer to:

- [Multi Environment Config Management](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config.ipynb)
